In [1]:
!pip install xgboost

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.tree import export_graphviz
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

from urllib.parse import urlparse,urlencode
import ipaddress
import re

# Phishing URL's

In [3]:
df = pd.read_csv('D:\\Software\\Anaconda\\ANA_Codes\\Project\\Data\\Main_Data_GS\\verified_online.csv')

In [4]:
df.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,7949592,https://angelicaig.cyou,http://www.phishtank.com/phish_detail.php?phis...,2022-11-18T09:52:34+00:00,yes,2022-11-18T10:03:36+00:00,yes,Other
1,7949586,https://attiva-bpercard.com/,http://www.phishtank.com/phish_detail.php?phis...,2022-11-18T09:19:44+00:00,yes,2022-11-18T09:33:21+00:00,yes,Other
2,7949581,https://screenpaste.cc/,http://www.phishtank.com/phish_detail.php?phis...,2022-11-18T09:14:00+00:00,yes,2022-11-18T09:33:21+00:00,yes,Other
3,7949577,http://sushiswapv3.com,http://www.phishtank.com/phish_detail.php?phis...,2022-11-18T09:08:50+00:00,yes,2022-11-18T09:13:17+00:00,yes,Other
4,7949578,https://sushiswapv3.com/,http://www.phishtank.com/phish_detail.php?phis...,2022-11-18T09:08:50+00:00,yes,2022-11-18T09:13:17+00:00,yes,Other


In [5]:
df.shape

(91569, 8)

In [6]:
# Taking 5000 url's randomly

df_phishrandom = df.sample(n=5000, random_state = 20).copy()
df_phishrandom.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
64254,7852269,http://www.mascorerd.yvey51.icu/page1.php,http://www.phishtank.com/phish_detail.php?phis...,2022-10-18T03:15:10+00:00,yes,2022-10-24T15:05:21+00:00,yes,NICOS
80057,7790262,http://itaupysecurei.byethost4.com/,http://www.phishtank.com/phish_detail.php?phis...,2022-09-26T15:03:56+00:00,yes,2022-09-26T15:23:51+00:00,yes,Other
58784,7858971,http://www.macesaeerd.tepyl0.icu/page1.php,http://www.phishtank.com/phish_detail.php?phis...,2022-10-18T12:25:15+00:00,yes,2022-10-24T08:13:21+00:00,yes,NICOS
44542,7883852,http://www.au-payacoy.accaomaoy.hfmu.top/AU/pa...,http://www.phishtank.com/phish_detail.php?phis...,2022-10-23T16:21:55+00:00,yes,2022-10-23T16:24:33+00:00,yes,KDDI
40209,7889854,http://www.eki-net.con-aesccoseeaas.gbaiwf.top...,http://www.phishtank.com/phish_detail.php?phis...,2022-10-25T06:07:25+00:00,yes,2022-10-25T06:10:04+00:00,yes,East Japan Railway Company


In [7]:
df_phishrandom.shape

(5000, 8)

In [8]:
df_phishrandom.isna().sum()

phish_id             0
url                  0
phish_detail_url     0
submission_time      0
verified             0
verification_time    0
online               0
target               0
dtype: int64

# Legitamate URL's

In [9]:
df1 = pd.read_csv('Data\\Main_Data_GS\\legi.csv')

In [10]:
df1.columns = ['URL']
df1.tail()

,URL
35372,https://lastpass.com/signup2.php?ac=1&from_uri...
35373,https://lastpass.com/signup2.php?ac=1&from_uri...
35374,https://lastpass.com/signup2.php?ac=1&from_uri...
35375,https://lastpass.com/signup2.php?ac=1&from_uri...
35376,https://asana.com/guide/videos/%22//fast.wisti...


In [11]:
df1.shape

(35377, 1)

In [12]:
# again taking 5000 website randomly

df_legirandom = df1.sample(n=5000, random_state = 10).copy()
df_legirandom.head()

,URL
13679,http://lifehacker.com/5900260/how-can-stop-wor...
6207,http://cookpad.com/recipe/list/212659?utf8=%E2...
1121,http://conservativetribune.com/civil-rights-le...
21020,http://distractify.com/igor-feng/28-photos-tha...
27573,http://motthegioi.vn/mao-trach-dong-qua-sach-b...


In [13]:
df_legirandom.shape

(5000, 1)

In [14]:
def getDomain(url):  
  domain = urlparse(url).netloc
  if re.match(r"^www.",domain):
	       domain = domain.replace("www.","")
  return domain

def havingIP(url):
  try:
    ipaddress.ip_address(url)
    ip = 1
  except:
    ip = 0
  return ip
def httpDomain(url):
  domain = urlparse(url).netloc
  if 'https' in domain:
    return 1
  else:
    return 0

In [15]:
def haveAtSign(url):
  if "@" in url:
    at = 1    
  else:
    at = 0    
  return at

def getLength(url):
  if len(url) < 54:
    length = 0            
  else:
    length = 1            
  return length
def prefixSuffix(url):
    if '-' in urlparse(url).netloc:
        return 1            
    else:
        return 0            

In [16]:
def getDepth(url):
  s = urlparse(url).path.split('/')
  depth = 0
  for j in range(len(s)):
    if len(s[j]) != 0:
      depth = depth+1
  return depth
def redirection(url):
  pos = url.rfind('//')
  if pos > 6:
    if pos > 7:
      return 1
    else:
      return 0
  else:
    return 0
def rightClick(response):
  if response == "":
    return 1
  else:
    if re.findall(r"event.button ?== ?2", response.text):
      return 0
    else:
      return 1

In [17]:
df_whole = pd.concat([df_legirandom, df_phishrandom]).reset_index(drop=True)

In [18]:
df_whole.to_csv('df_whole.csv', index=False)